# 1 Import Required Modules

In [1]:
import matplotlib.pyplot as plt 
import numpy as np              
import time                     
import random                   
import copy

import torch
import torchvision
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

!pip install torchinfo
from torchinfo import summary

torch.cuda.empty_cache()

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

# 2 Prepare Data

We will use CIFAR-10 dataset in order to test CPLI method which can be read from [here](https://arxiv.org/pdf/2003.06757.pdf). In the original paper, batch size is mentioned as 256. So we will use this values as batch size. But firstly a function for creating batches for a certain batch_size will be defined. 

In [2]:
def create_batches(batchSize):
  # Make required transformation which is necessary for the inputs to VGG19 network
  # Further info can be accessed from https://pytorch.org/hub/pytorch_vision_vgg/ 
  TF = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=TF)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchSize,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=TF)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batchSize,
                                          shuffle=False, num_workers=2)
  return trainloader, testloader

CIFAR10_classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## 2.1 Enable GPU

From "Edit -> Notebook Settings -> Hardware accelerator" select GPU. With the following we will specify to PyTorch that we want to use the GPU.

In [3]:
if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available :(")
  device = torch.device("cpu")

Cuda (GPU support) is available and enabled!


# 3 Download, Finetune and Test Original Pretrained VGG19

In [4]:
# Create an instance of original pretrained VGG19
origVGG = torchvision.models.vgg19(pretrained=True)
# Visualize the network.
print(origVGG)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

## 3.1 Finetune Pre-Trained VGG19 on CIFAR-10

Since VGG19 is pretrained on ImageNet dataset, we have to finetune the network for CIFAR-10.

In [5]:
# Freeze the layers by setting requires_grad parameter to False.
newVGG = copy.deepcopy(origVGG)
for param in newVGG.parameters():
  param.requires_grad = False

In [6]:
newVGG.classifier[6] = None
newVGG.classifier[6] = nn.Linear(4096, 10)
print(newVGG.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)


Define a $\textbf{train}$ function in order to train the network

In [7]:
def train(model, criterion, optimizer, epochs, dataloader, verbose=True):
  """
    Define the trainer function. We can use this for training any model.
    The parameter names are self-explanatory.

    Returns: the loss history.
  """
  loss_history = [] 
  for epoch in range(epochs):
    for i, data in enumerate(dataloader, 0):    
      
      # Our batch:
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)

      # zero the gradients as PyTorch accumulates them
      optimizer.zero_grad()

      # Obtain the scores
      outputs = model(inputs)

      # Calculate loss
      loss = criterion(outputs.to(device), labels)

      # Backpropagate
      loss.backward()

      # Update the weights
      optimizer.step()

      loss_history.append(loss.item())
    
    if verbose: print(f'Epoch {epoch} / {epochs}: avg. loss of last 5 iterations {np.sum(loss_history[:-6:-1])/5}')

  return loss_history

Create the learnable parameters and make those the parameter of SGD optimizer. Also create the instances of loss function and send the model to GPU.

In [8]:
def get_learnable_parameters(model):
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
    return params_to_update

batch_finetune = 1024
trainloader, testloader = create_batches(batch_finetune)
weight_decay = 0.0001
parameters_to_update = get_learnable_parameters(newVGG)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(parameters_to_update, lr=0.1, momentum=0.9, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer,1)

newVGG = newVGG.to(device)
epochs = 5
loss_history = train(newVGG, criterion, optimizer, epochs, trainloader)

Files already downloaded and verified
Files already downloaded and verified


RuntimeError: ignored

In [ ]:
 torch.cuda.memory_summary()